# GBT-PWV01b : Explore Features forPWV seletion

- author Sylvie Dagoret-Campagne
- creation date 2026-01-27 : version run2026_v01 use HistGradientBoosting
- last update : 2026-01-30 : must keep early stopping and do correctly the trainning
- affiliation : IJCLab
- Kernel @usdf **w_2026_02*
- Home emac : base (conda)
- laptop : conda_py313

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from platform import python_version
print(python_version())

In [ ]:
import warnings
warnings.resetwarnings()
warnings.simplefilter('ignore')

In [ ]:
# must install the mysitcom package by doing at top level "pip install --user -e . "
from mysitcom.auxtel.pwv import scatter_datetime
from mysitcom.auxtel.pwv import strip_datetime
from mysitcom.auxtel.pwv import bar_counts_by_night
from mysitcom.auxtel.pwv import plot_dccd_chi2_vs_time
from mysitcom.auxtel.pwv import plot_dccd_chi2_vs_time_by_filter
from mysitcom.auxtel.pwv import stripplot_target_vs_time
from mysitcom.auxtel.pwv import plot_dccd_chi2_vs_time_by_target_filter
from mysitcom.auxtel.pwv import plot_dccd_chi2_histo_by_target_filter
from mysitcom.auxtel.pwv import plot_dccd_chi2_vs_time_by_target_filter_colorsedtype
from mysitcom.auxtel.pwv import plot_dccd_chi2_histo_by_target_filter_colorsedtype
from mysitcom.auxtel.pwv import summarize_dccd_chi2
from mysitcom.auxtel.pwv import plot_atmparam_vs_time, plot_atmparam_diff_vs_time
from mysitcom.auxtel.pwv import plot_atmparam_hist_per_filter, plot_atmparam_diff_hist_per_filter

from mysitcom.auxtel.pwv import GetNightMidnightsDict,GetNightBoundariesDict

In [ ]:
from mysitcom.auxtel.gradientboosttree import normalize_column_data_bytarget_byfilter

In [ ]:
import os

In [ ]:
# where are stored the figures
pathfigs = "figs_GBT-PWV01b"
prefix = "gbt-pwv01b"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs) 
figtype = ".png"

In [ ]:
import numpy as np
from numpy.linalg import inv
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm
from matplotlib.patches import Circle,Annulus
from astropy.visualization import ZScaleInterval
props = dict(boxstyle='round', facecolor="white", alpha=0.1)
#props = dict(boxstyle='round')

import matplotlib.colors as colors
import matplotlib.cm as cmx

import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from matplotlib.gridspec import GridSpec

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.wcs import WCS
from astropy import units as u
from astropy import constants as c

from scipy import interpolate
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KDTree, BallTree

import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd
from pandas.api.types import is_datetime64_any_dtype

import pickle
from collections import OrderedDict

plt.rcParams["figure.figsize"] = (16,8)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams["legend.fontsize"] = "xx-large"

import scipy
from scipy.optimize import curve_fit,least_squares

from pprint import pprint

# new color correction model
import pickle
from scipy.interpolate import RegularGridInterpolator

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import shap

In [ ]:
# Remove to run faster the notebook
#import ipywidgets as widgets
#%matplotlib widget

In [ ]:
from GBT_PWV00_parameters import *

In [ ]:
DumpConfig()

In [ ]:
from importlib.metadata import version

In [ ]:
# wavelength bin colors
#jet = plt.get_cmap('jet')
#cNorm = mpl.colors.Normalize(vmin=0, vmax=NSED)
#scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
#all_colors = scalarMap.to_rgba(np.arange(NSED), alpha=1)

In [ ]:
np.__version__

In [ ]:
pd.__version__

### Configuration

In [ ]:
def convertNumToDatestr(num):
    year = num//10_000
    month= (num-year*10_000)//100
    day = (num-year*10_000-month*100)

    year_str = str(year).zfill(4)
    month_str = str(month).zfill(2)
    day_str = str(day).zfill(2)
    
    datestr = f"{year_str}-{month_str}-{day_str}"
    return pd.to_datetime(datestr)

In [ ]:
PWVMIN = 0.
PWVMAX = 20.

In [ ]:
FLAG_WITHCOLLIMATOR = False
DATE_WITHCOLLIMATOR = 20230930
datetime_WITHCOLLIMATOR = convertNumToDatestr(DATE_WITHCOLLIMATOR)
datetime_WITHCOLLIMATOR = pd.to_datetime("2023-09-30 00:00:00.0+0000")
datetime_WITHCOLLIMATOR

## Initialisation

### Read the file
- `atmfilename` is defined in `PWV00_parameters.py` 

In [ ]:
the_suptitle = butlerusercollectiondict[version_run] 

In [ ]:
inputfilename = atmfilename.split("/")[-1]

if "parquet" in inputfilename:
    df_spec = pd.read_parquet(atmfilename)
elif "npy" in inputfilename:
    specdata = np.load(atmfilename,allow_pickle=True)
    df_spec = pd.DataFrame(specdata)
    df_spec["D_CCD [mm]"] = df_spec["D2CCD"]
    df_spec["PWV [mm]"] = df_spec["PWV [mm]_x"] 
    df_spec["PWV [mm]_rum"] = df_spec["PWV [mm]_y"] 
    df_spec["PWV [mm]_err"] = df_spec["PWV [mm]_err_x"] 
    df_spec["PWV [mm]_err_rum"] = df_spec["PWV [mm]_err_y"] 


    cols = [
    "PWV [mm]",
    "PWV [mm]_rum",
    "PWV [mm]_err",
    "PWV [mm]_err_rum",
    ]

    df_spec = df_spec.dropna(subset=cols)
    
else:
    raise "bad path of filename {inputfilename}"
    

In [ ]:
print(" | ".join(df_spec.columns)) 

In [ ]:
#df_spec.dtypes.to_frame('Type de donnée')

In [ ]:
# add time for plotting
#df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"])
df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"],utc=True)

In [ ]:
df_spec["nightObs"] = df_spec.apply(lambda x: x['id']//100_000, axis=1)

In [ ]:
df_spec["seq_num"]  = df_spec["id"] % 100_000

## Select only empty and OG550 filters

In [ ]:
df_spec["FILTER"].unique()

In [ ]:
if FLAG_PWVFILTERS: 
    df_spec = df_spec[df_spec["FILTER"].isin(PWV_FILTER_LIST) ]

## Special study on Star Color (Spectral type)
- Load magnitudes from external file
- the magnitudes have been computed in another notebooks in ../2025-10-29-TOOLS



In [ ]:
targets_mag_files = "../2025-10-29-TOOLS/data/targets_magnitudes.csv"
df_targets_mag = pd.read_csv(targets_mag_files,index_col=0)      
df_targets_mag = df_targets_mag.sort_values(by="y")

### palette with SED type

In [ ]:
df_col = df_targets_mag.copy()
df_col = df_col.sort_values(by="B_V")

SpT = df_col["Sp_T"].values
unique_types = list(dict.fromkeys(SpT))  # garde l'ordre d'apparition
N_types = len(unique_types)


# Associe chaque type spectral à un entier
type_to_idx = {t: i for i, t in enumerate(unique_types)}
idx = np.array([type_to_idx[t] for t in SpT])

# Crée la colormap
cmap = mpl.cm.jet
norm = mpl.colors.Normalize(vmin=-0.5, vmax=N_types - 0.5)

# Colorbar horizontale
fig, ax = plt.subplots(figsize=(14, 0.4), layout="constrained")
cbar = fig.colorbar(
    mpl.cm.ScalarMappable(norm=norm, cmap=cmap),
    cax=ax,
    orientation="horizontal",
    ticks=range(N_types),
)
cbar.ax.set_xticklabels(unique_types, rotation=45, ha="right")
cbar.set_label("Spectral Type", fontsize=12,labelpad=10)

plt.show()

In [ ]:
df_targets_mag.head() 

### Target ordered by colors in the input file

In [ ]:
sorted_targets =  list(df_targets_mag.index) 
print(sorted_targets)

### Target ordered by colors in our data  file

In [ ]:
# order pf selected target by magnitude Y
order_selected_targets = [t for t in sorted_targets if t in df_spec["TARGET"].unique()]
print(order_selected_targets)

### build a palette for colors

In [ ]:
# --- Palette personnalisée cohérente avec la colormap jet ---
target_to_color = {}
for target in order_selected_targets:
    sp_type = df_targets_mag.loc[target, "Sp_T"]
    rgba = cmap(norm(type_to_idx[sp_type]))

    # Convert to pure Python floats
    target_to_color[target] = tuple(float(c) for c in rgba)

# Plot PWV vs time before application cuts

### Calculate difference and ratio for some params

In [ ]:
denom = np.sqrt(df_spec["PWV [mm]_err"]**2 + df_spec["PWV [mm]_err_rum"]**2)

df_spec["diff_PWV_norm"] = np.where(
    np.isfinite(denom) & (denom > 0),
    (df_spec["PWV [mm]"] - df_spec["PWV [mm]_rum"]) / denom,
    np.nan
)

df_spec["diff_PWV"] =  (df_spec["PWV [mm]"] - df_spec["PWV [mm]_rum"]) 
df_spec["diff_PWV_err"] = np.sqrt( (df_spec["PWV [mm]_err"]**2 - df_spec["PWV [mm]_err_rum"]**2)) 

## Calculate midnights and night boundaries

In [ ]:
DT = pd.Timedelta(minutes=7*24*60)
TMIN  = df_spec["Time"].min()-DT
TMAX  = df_spec["Time"].max()+DT

In [ ]:
# get night boundaries
dn = GetNightBoundariesDict(df_spec)
# get midnights
dnidnights = GetNightMidnightsDict(df_spec)

### Plot PWV in spectrogram vs Time before quality cuts

In [ ]:
fig,ax = plot_atmparam_vs_time(
    df_spec,
    time_col= "Time",
    filter_col = "FILTER",
    param_col = "PWV [mm]",
    param_err_col = "PWV [mm]_err",
    title_param = "PWV vs time (spectrogram no cut qual. cut)",
    
    # seuils / bornes
    param_min_fig=PWVMIN,
    param_max_fig=PWVMAX,
    param_min_cut=None,
    param_max_cut=None,
 
    # titres
    suptitle= the_suptitle,

    # axes externes
    axs=None,
    figsize=(18, 6),
)

if version_run not in ["run_v12"]:
    for key, tt in dn.items():
        ax.axvspan(tt[0],tt[1], color='blue', alpha=0.05,lw=0.5)

for key, midn in dnidnights.items():
    ax.axvline( midn ,color="purple",ls=":",alpha=0.5,lw=0.5)

### Plot PWV in spectrum vs Time

In [ ]:
fig,ax = plot_atmparam_vs_time(
    df_spec,
    time_col= "Time",
    filter_col = "FILTER",
    param_col = "PWV [mm]_rum",
    param_err_col = "PWV [mm]_err_rum",
    title_param = "PWV vs time (spectrum no cut qual. cut)",
    
    # seuils / bornes
    param_min_fig=PWVMIN,
    param_max_fig=PWVMAX,
    param_min_cut=None,
    param_max_cut=None,
 
    # titres
    suptitle= the_suptitle,

    # axes externes
    axs=None,
    figsize=(18, 6),
)

if version_run not in ["run_v12"]:
    for key, tt in dn.items():
        ax.axvspan(tt[0],tt[1], color='blue', alpha=0.05,lw=0.5)

for key, midn in dnidnights.items():
    ax.axvline( midn ,color="purple",ls=":",alpha=0.5,lw=0.5)

### Plot PWV_spectrogram - PWV_spectrum vs Time before quality cut

In [ ]:
fig,ax = plot_atmparam_vs_time(
    df_spec,
    time_col= "Time",
    filter_col = "FILTER",
    param_col = "diff_PWV",
    param_err_col = "diff_PWV_err",
    title_param = "$\Delta$ PWV vs time (spectrogram - spectrum no cut qual. cut)",
    
    # seuils / bornes
    param_min_fig=-PWVMAX/10.,
    param_max_fig=PWVMAX/10.,
    param_min_cut=None,
    param_max_cut=None,
 
    # titres
    suptitle= the_suptitle,

    # axes externes
    axs=None,
    figsize=(18, 6),
)

if version_run not in ["run_v12"]:
    for key, tt in dn.items():
        ax.axvspan(tt[0],tt[1], color='blue', alpha=0.05,lw=0.5)

for key, midn in dnidnights.items():
    ax.axvline( midn ,color="purple",ls=":",alpha=0.5,lw=0.5)

In [ ]:
fig,ax = plot_atmparam_hist_per_filter(
    df_spec,
    filter_col="FILTER",
    param_col = "diff_PWV",
    param_range = (-PWVMAX/10.,PWVMAX/10.),

    # histogram control
    bins=100,
    density=True,
    hist_alpha=0.4,

    # x-axis limits
    param_min_fig=-PWVMAX/10,
    param_max_fig=PWVMAX/10.,

    title_param="$\Delta$ PWV vs time (spectrogram - spectrum no cut qual. cut)",
    # titres
    suptitle= the_suptitle
)
plt.show()

# Regression fit 

## 1) Preprocess the data

In [ ]:
df_spec, df1 = normalize_column_data_bytarget_byfilter(df_spec,target_col="TARGET",filter_col="FILTER",feature_col= "CHI2_FIT",ext="norm")
df_spec, df2 = normalize_column_data_bytarget_byfilter(df_spec,target_col="TARGET",filter_col="FILTER",feature_col= "chi2",ext="norm")
df_spec, df3 = normalize_column_data_bytarget_byfilter(df_spec,target_col="TARGET",filter_col="FILTER",feature_col= "chi2_rum",ext="norm")

In [ ]:
df_spec.head()

In [ ]:
df_spec.tail()

## 3️⃣ Define the quality label (Y)
- You already suggested the right idea.
- Recommended target variable
- Use continuous first:

In [ ]:
df_spec["abs_delta_PWV"] = np.abs(df_spec["PWV [mm]"] - df_spec["PWV [mm]_rum"])

In [ ]:
q = df_spec["abs_delta_PWV"].quantile(0.95)   # or 0.95
df_spec["bad"] = df_spec["abs_delta_PWV"] > q

## 4️⃣ Feature hygiene (critical step)
- ❌ Remove forbidden features
- ❌ Remove identifiers
- ✔️ Keep physically meaningful predictors

- Remove unnormalised chi2
- categoy like taget, filter

In [ ]:
columns_keep = ["id","Time","TARGET","ROTANGLE","D2CCD", "DOMEAZ","AZ","EL","WINDSPD", "WINDDIR","PARANGLE","TARGETX","TARGETY","CHI2_FIT_norm","PIXSHIFT","PSF_REG","TRACE_R", 
"A2_FIT", "AM_FIT", "MEANFWHM", "AIRMASS", "OUTTEMP", "OUTPRESS", "OUTHUM","FILTER", "CAM_ROT","chi2_norm","A1", "A2", "A3", "PWV [mm]" ,"B",
"A_star","D_CCD [mm]","shift_x [pix]","shift_y [pix]", "angle [deg]", "P [hPa]","gamma_0_1", "gamma_1_1","gamma_2_1", "alpha_0_1","alpha_1_1","saturation_0_1",
"gamma_0_2","gamma_1_2","gamma_2_2", "alpha_0_2", "alpha_1_2", "alpha_2_2", "saturation_0_2", "chi2_rum_norm", "A1_rum", "A2_rum",
"PWV [mm]_rum", "reso [nm]", "D_CCD [mm]_rum", "alpha_pix [pix]", "mount_motion_image_degradation_x",
"mount_motion_image_degradation_az_x", "mount_motion_image_degradation_el_x", "mount_jitter_rms_x","mount_jitter_rms_az_x", "mount_jitter_rms_el_x", "mount_jitter_rms_rot_x",
"dimm_seeing_x", "focus_z_x" ,"mount_motion_image_degradation_y", "mount_motion_image_degradation_az_y","diff_PWV","abs_delta_PWV","bad"]

#### First keep for empty filter

In [ ]:
df_spec = df_spec[columns_keep]

In [ ]:
filter_sel = "empty"

In [ ]:
df_spec_sel = df_spec[df_spec["FILTER"] == filter_sel]
N = len(df_spec_sel)
print(f"N = {N}")

In [ ]:
#df_Y = df_spec_sel["abs_delta_PWV"]
#Y = df_spec_sel["abs_delta_PWV"].values

df_Y = df_spec_sel["diff_PWV"]
Y = df_spec_sel["diff_PWV"].values

NY = len(Y)
print(f"NY = {NY}")

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,4))
ax.hist(Y,bins=50)
ax.set_yscale("log")
ax.set_xlabel("Y")
ax.set_ylabel("N")
plt.show()

In [ ]:
columns_todropforX =  ["id","Time","TARGET","FILTER","PWV [mm]","PWV [mm]_rum","diff_PWV","abs_delta_PWV","bad","OUTHUM",
                      "mount_motion_image_degradation_x",     
                        "mount_motion_image_degradation_az_x",
                        "mount_motion_image_degradation_el_x",    
                        "mount_jitter_rms_x",                     
                        "mount_jitter_rms_az_x",                  
                        "mount_jitter_rms_el_x",                  
                        "mount_jitter_rms_rot_x",                 
                        "dimm_seeing_x",                          
                        "focus_z_x",                                 
                        "mount_motion_image_degradation_y",       
                        "mount_motion_image_degradation_az_y"]    

In [ ]:
df_X = df_spec_sel.drop(columns = columns_todropforX) 

In [ ]:
# Check the number of nan
nan_per_col = df_X.isna().sum()
print(nan_per_col)

In [ ]:
df_X.dtypes.value_counts()

In [ ]:
# print object columns
obj_cols = df_X.select_dtypes(include="object").columns
print(obj_cols)

- lines (samples)

In [ ]:
len(df_X)

- Columns (features)

In [ ]:
len(df_X.columns)

# 2) Fit a model HistGradientBoostingRegressor with `early_stopping  = True` --> model1

Voici un résumé stratégique pour configurer votre HistGradientBoostingRegressor. 

Ces paramètres ne travaillent pas isolément, ils forment un équilibre entre puissance du modèle et capacité de généralisation.

1. )Structure de l'Arbre (Complexité)


`max_leaf_nodes` (Le plus important) :
C'est le principal levier de complexité. Il limite le nombre de décisions par arbre.Conseil : 31 est le standard. Augmentez (63, 127) pour des données complexes, réduisez (15, 20) si vous avez peu de données.

`max_depth` : Limite la distance maximale entre la racine et une feuille.
Conseil : Dans cet algorithme, on le laisse souvent à None ou à une valeur élevée (20+), car max_leaf_nodes fait déjà le travail de limitation.

`min_samples_leaf` : Le garde-fou. Il impose un nombre minimum d'individus dans chaque feuille.Conseil : Avec 7000 lignes, le défaut (20) est bien. Augmentez-le si vous voyez du surapprentissage (overfitting) malgré un faible nombre de feuilles.


2.) Processus d'Apprentissage (Vitesse et Précision)

`learning_rate` : La taille des pas que fait le modèle à chaque nouvel arbre.Conseil : Une valeur faible (0.01) est plus précise mais demande plus d'arbres. Une valeur élevée (0.1) est plus rapide mais peut "rater" l'optimum.

`max_iter` : Le nombre maximum d'arbres à construire.Conseil : Avec un learning_rate de 0.01, visez 1000 à 2000. C'est le early_stopping qui décidera de la fin réelle.


3.) Contrôle de l'Arrêt (Stabilité)

`early_stopping` : Active l'arrêt automatique dès que le modèle ne progresse plus. Indispensable pour gagner du temps et éviter le surapprentissage.

`validation_fraction` : La portion de données (ex: 0.1 pour 10%) "mise de côté" pour tester le modèle à chaque itération.Conseil : Avec 7000 lignes, 0.1 ou 0.15 suffisent. Vos 0.3 actuels sont peut-être un peu élevés (vous privez le modèle de trop de données d'entraînement).

`n_iter_no_change` : La "patience". Combien d'itérations le modèle attend-il sans amélioration avant de s'arrêter ? Conseil : 10 est standard. Si votre courbe de score est très "hachée", montez à 20.


Synthèse : 

- Le compromis Idéal 

Objectif Réglage : Éviter le surapprentissage
Baisser `max_leaf_nodes`, monter `min_samples_leaf`

- Améliorer la précision : Baisser `learning_rate`, monter `max_iter`.

- Réduire le temps de calcul:
Utiliser `early_stopping`, monter `learning_rate`.

1. Le Sous-apprentissage (Underfitting)

Le modèle est trop simple pour capturer les relations dans les données. Il "rate" sa cible sur les deux tableaux.Signe : 

Erreur élevée en Train ET erreur élevée en Validation.Analogie : Un élève qui n'a pas assez étudié et qui échoue à la fois aux exercices de révision et à l'examen final.Causes possibles : `max_depth trop petit`, `max_leaf_nodes` trop bas, ou `max_iter` insuffisant.


2. Le Sur-apprentissage (Overfitting)

Le modèle est trop complexe et "mémorise" le bruit et les cas particuliers au lieu d'apprendre des règles générales.Signe : Erreur très faible en Train MAIS erreur élevée (ou qui remonte) en Validation.

Analogie : Un élève qui a appris par cœur les annales de l'examen mais qui est incapable de répondre si on change un seul chiffre dans l'énoncé.Causes possibles : max_depth trop élevé, trop de max_leaf_nodes, ou learning_rate trop fort sans early_stopping.



- Si les deux courbes stagnent bas : Vous êtes en sous-apprentissage. Solution : Augmentez max_leaf_nodes.

- Si la courbe Train continue de monter mais que la courbe Validation redescend ou stagne : Vous êtes en sur-apprentissage. Solution : Diminuez max_depth ou augmentez min_samples_leaf.

- Si les deux courbes sont proches et hautes : C'est le "Sweet Spot" (l'équilibre idéal).

    model_robust = HistGradientBoostingRegressor(
    max_iter=2000,
    learning_rate=0.01,
    
    # --- On bride la complexité ---
    max_leaf_nodes=20,           # Moins de feuilles
    min_samples_leaf=50,         # Plus de monde par feuille
    l2_regularization=1.0,       # On ajoute une pénalité
    
    # --- On durcit l'arrêt ---
    early_stopping=True,
    n_iter_no_change=15,         # On arrête plus vite si ça ne progresse plus
    validation_fraction=0.15,
    random_state=42
    )

In [ ]:
HGB1_MAX_DEPTH = 20
HGB1_LEARNING_RATE = 0.02
HGB1_MAX_ITER = 1000
#HGB1_MAX_LEAF_NODES = 15
HGB1_MAX_LEAF_NODES = 63
HGB1_VALIDATION_FRACTION = 0.3
HGB1_NITER_NOCHANGE = 20
HGB1_MIN_SAMPLE_LEAF = 20
suptitle1 = f"HistGradientBoostingRegressor:: max_dep = {HGB1_MAX_DEPTH}, lr={HGB1_LEARNING_RATE},val_frac={HGB1_VALIDATION_FRACTION},early stop"

In [ ]:
HGB1_MAX_DEPTH = 20
HGB1_LEARNING_RATE = 0.05
HGB1_MAX_ITER = 500
HGB1_MAX_LEAF_NODES = 31
HGB1_VALIDATION_FRACTION = 0.2
HGB1_NITER_NOCHANGE = 15
HGB1_MIN_SAMPLE_LEAF = 30
HGB1_L2_REGULARISATION = 1.0
suptitle1 = f"HistGradientBoostingRegressor:: max_dep = {HGB1_MAX_DEPTH}, lr={HGB1_LEARNING_RATE},val_frac={HGB1_VALIDATION_FRACTION},early stop"

- Note : learning_rate × max_iter ≈ 5 – 20

In [ ]:
HGB1_LEARNING_RATE * HGB1_MAX_ITER 

#### Simple training

In [ ]:
model1 = HistGradientBoostingRegressor(
    max_depth       = HGB1_MAX_DEPTH,
    max_leaf_nodes  = HGB1_MAX_LEAF_NODES,
    min_samples_leaf = HGB1_MIN_SAMPLE_LEAF,
    learning_rate   = HGB1_LEARNING_RATE,
    max_iter        = HGB1_MAX_ITER,
    early_stopping  = True,
    validation_fraction = HGB1_VALIDATION_FRACTION,
    n_iter_no_change    = HGB1_NITER_NOCHANGE,
    l2_regularization = HGB1_L2_REGULARISATION, 
    random_state=42
)
model1.fit(df_X, df_Y)

In [ ]:
# Récupération du nombre d'itérations
print(f"Optimal Nb iterations : {model1.n_iter_}")

In [ ]:
print(f"model.train_score_      = {model1.train_score_[-1]}")     # loss sur train
print(f"model.validation_score_ = {model1.validation_score_[-1]}")  # loss sur validation

In [ ]:
# compute mean squared error
mse = []
for y_pred in model1.staged_predict(df_X):
    mse.append(mean_squared_error(df_Y, y_pred))

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(14,5),constrained_layout=True)
ax1,ax2 = axs
ax1.plot(mse,color='b')
ax1.set_xlabel("Iteration")
ax1.set_ylabel("MSE")
ax1.set_title("MSE")
ax1.grid()

ax2.plot(model1.train_score_,color='b',label="model.train_score_"  )
ax2.plot(model1.validation_score_ ,color="r",label="model.validation_score_ "  )
ax2.set_xlabel("Iteration")
ax2.set_ylabel("score")
ax2.set_title("Score")
ax2.grid()
ax2.legend()
plt.suptitle(suptitle1)
plt.show()


In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,5),constrained_layout=True)
ax.scatter(df_Y,y_pred,alpha=0.1)
ax.set_xlabel("$Y_{true}$")
ax.set_ylabel("$Y_{pred}$")
ax.plot([-20,20],[-20,20],lw=1,color="k")
ax.set_aspect("equal")
ax.grid()
ax.set_title(suptitle1)

#### Iteration on `max_depth`

In [ ]:
depths_to_test = [5, 10, 20, 50, None] # None = profondeur illimitée
results = []

for depth in depths_to_test:
    model = HistGradientBoostingRegressor(
        max_depth=depth,
        max_leaf_nodes  = HGB1_MAX_LEAF_NODES,
        min_samples_leaf = HGB1_MIN_SAMPLE_LEAF,
        learning_rate=HGB1_LEARNING_RATE,
        max_iter=HGB1_MAX_ITER,
        early_stopping=True,
        validation_fraction=HGB1_VALIDATION_FRACTION,
        n_iter_no_change=HGB1_NITER_NOCHANGE,
        l2_regularization = HGB1_L2_REGULARISATION, 
        random_state=42
    )
    
    model.fit(df_X, df_Y)
    
    # On récupère le dernier score de validation (le meilleur)
    # Note: validation_score_ contient les scores à chaque itération
    best_val_score = model.validation_score_[-1]
    best_train_score = model.train_score_[-1]
    results.append({
        'max_depth': depth if depth is not None else "Unlimited",
        'best_train_score': best_train_score,
        'best_val_score': best_val_score,
        'iterations_needed': model.n_iter_
    })

# Affichage des résultats
df_results = pd.DataFrame(results)
print(df_results)

#### Iteration on `max_leaf_nodes`

In [ ]:
leaf_nodes_to_test = [15, 31, 63, 127]
results = []

for nodes in leaf_nodes_to_test:
    model = HistGradientBoostingRegressor(
        max_depth  = HGB1_MAX_DEPTH, # On fixe la profondeur gagnante
        max_leaf_nodes = nodes,
        min_samples_leaf = HGB1_MIN_SAMPLE_LEAF,
        learning_rate = HGB1_LEARNING_RATE,
        max_iter= HGB1_MAX_ITER,
        early_stopping=True,
        validation_fraction = HGB1_VALIDATION_FRACTION,
        n_iter_no_change = HGB1_NITER_NOCHANGE,
        l2_regularization = HGB1_L2_REGULARISATION, 
        random_state=42
    )
    model.fit(df_X, df_Y)
    results.append({'nodes': nodes, 'score': model.validation_score_[-1]})

In [ ]:
print(pd.DataFrame(results))

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [10, 20],
    'max_leaf_nodes': [15, 31, 63, 127] # Très important à coupler avec max_depth
}

grid = GridSearchCV(model1, param_grid, cv=5,n_jobs=-1)
grid.fit(df_X, df_Y)

In [ ]:
print(f"GridSearchCV :: best max_depth : {grid.best_params_['max_depth']}")
print(f"GridSearchCV :: best max_leaf_nodes : {grid.best_params_['max_leaf_nodes']}")

In [ ]:
from sklearn.experimental import enable_halving_search_cv  # Obligatoire pour l'instant
from sklearn.model_selection import HalvingGridSearchCV

param_grid = {
    'max_depth': [10, 20],
    'max_leaf_nodes': [15, 31, 63, 127] # Très important à coupler avec max_depth
}

# C'est beaucoup plus rapide pour les gros modèles de boosting
grid = HalvingGridSearchCV(model1, param_grid, cv=5, factor=2, resource='n_samples')
grid.fit(df_X, df_Y)

In [ ]:
print(f"HalvingGridSearchCV :: best max_depth : {grid.best_params_['max_depth']}")
print(f"HalvingGridSearchCV :: best max_leaf_nodes : {grid.best_params_['max_leaf_nodes']}")

#### Final model

In [ ]:
model1 = HistGradientBoostingRegressor(
    max_depth       = HGB1_MAX_DEPTH,
    max_leaf_nodes  = HGB1_MAX_LEAF_NODES,
    min_samples_leaf = HGB1_MIN_SAMPLE_LEAF,
    learning_rate   = HGB1_LEARNING_RATE,
    max_iter        = HGB1_MAX_ITER,
    early_stopping  = True,
    validation_fraction = HGB1_VALIDATION_FRACTION,
    n_iter_no_change    = HGB1_NITER_NOCHANGE,
    l2_regularization = HGB1_L2_REGULARISATION, 
    random_state=42
)

model1.fit(df_X, df_Y)

In [ ]:
# Récupération du nombre d'itérations
print(f"Optimal Nb iterations   = {model1.n_iter_}")
print(f"model.train_score_      = {model1.train_score_[-1]}")     # loss sur train
print(f"model.validation_score_ = {model1.validation_score_[-1]}")  # loss sur validation

### compute mean squared error after fitting

In [ ]:
# compute mean squared error
mse = []
for y_pred in model1.staged_predict(df_X):
    mse.append(mean_squared_error(df_Y, y_pred))

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,5),constrained_layout=True)
ax.scatter(df_Y,y_pred,alpha=0.1)
ax.set_xlabel("$Y_{true}$")
ax.set_ylabel("$Y_{pred}$")
ax.plot([-20,20],[-20,20],lw=1,color="k")
ax.set_aspect("equal")
ax.grid()
ax.set_title(suptitle1)

### Plot fit control

In [ ]:
fig,axs = plt.subplots(1,2,figsize=(14,5),constrained_layout=True)
ax1,ax2 = axs
ax1.plot(mse,color='b')
ax1.set_xlabel("Iteration")
ax1.set_ylabel("MSE")
ax1.set_title("MSE")
ax1.grid()

ax2.plot(model1.train_score_,color='b',label="model.train_score_"  )
ax2.plot(model1.validation_score_ ,color="r",label="model.validation_score_ "  )
ax2.set_xlabel("Iteration")
ax2.set_ylabel("score")
ax2.set_title("Score")
ax2.grid()
ax2.legend()
plt.suptitle(suptitle1)
plt.show()


In [ ]:
# Récupération du nombre d'itérations
print(f"Optimal Nb iterations   = {model1.n_iter_}")
print(f"model.train_score_      = {model1.train_score_[-1]}")     # loss sur train
print(f"model.validation_score_ = {model1.validation_score_[-1]}")  # loss sur validation

In [ ]:
#print(f"model.train_score_      = {model1.train_score_}")     # loss sur train
#print(f"model.validation_score_ = {model1.validation_score_}")  # loss sur validation

## 3) Check the Fit with a model HistGradientBoostingRegressor with `early_stopping  = False` --> model2

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_X, df_Y, test_size=0.3, random_state=42,shuffle=True)
#X_train, X_test, y_train, y_test = train_test_split(df_X, df_Y, test_size=0.2, random_state=42,shuffle=False)

In [ ]:
HGB2_MAX_DEPTH = HGB1_MAX_DEPTH
HGB2_LEARNING_RATE = HGB1_LEARNING_RATE
HGB2_MAX_ITER = HGB1_MAX_ITER
HGB2_NITER_NOCHANGE = HGB1_NITER_NOCHANGE
HGB2_MAX_LEAF_NODES = HGB1_MAX_LEAF_NODES
HGB2_MIN_SAMPLE_LEAF = HGB1_MIN_SAMPLE_LEAF
HGB2_L2_REGULARISATION = HGB1_L2_REGULARISATION 
suptitle2 = f"HistGradientBoostingRegressor:: max_dep = {HGB2_MAX_DEPTH}, lr={HGB2_LEARNING_RATE}, no early stop"

In [ ]:
HGB2_ITER_STEP = 10

### Where is the iteration stop determined with early_stop

In [ ]:
train_mse = []
test_mse = []
iter_num = []
models_history = {}
for n in range(1, HGB2_MAX_ITER, HGB2_ITER_STEP):
    model2 = HistGradientBoostingRegressor(
        max_depth       = HGB2_MAX_DEPTH,
        max_leaf_nodes  = HGB2_MAX_LEAF_NODES,
        min_samples_leaf = HGB2_MIN_SAMPLE_LEAF,
        learning_rate   = HGB2_LEARNING_RATE ,
        l2_regularization = HGB2_L2_REGULARISATION, 
        max_iter        = n ,
        early_stopping=False,
        random_state=42
    )
    
    model2.fit(X_train, y_train)
    models_history[n] = model2

    y_train_pred = model2.predict(X_train)
    y_test_pred = model2.predict(X_test)

    train_mse.append(mean_squared_error(y_train, y_train_pred))
    test_mse.append(mean_squared_error(y_test, y_test_pred))
    iter_num.append(n)


In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,5),constrained_layout=True)

ax.plot(iter_num,train_mse,color='b',label="train_mean_squared_error"  )
ax.plot(iter_num,test_mse ,color="r",label="test_mean_squared_error"  )
ax.set_xlabel("Iteration")
ax.set_ylabel("training error")
ax.set_title("training error")
ax.grid()
ax.legend()
ax.axvline(model1.n_iter_,c="k")
plt.suptitle(suptitle2)
plt.show()


### Final loop on iteration

- find myself the best iteration

In [ ]:
# 1. Initialisation
models_history = {}
train_mse = []
test_mse = []
iter_num = []

# 2. La Boucle (votre code optimisé)
for n in range(1, HGB2_MAX_ITER, HGB2_ITER_STEP):
    model = HistGradientBoostingRegressor(
        max_depth=HGB2_MAX_DEPTH,
        max_leaf_nodes=HGB2_MAX_LEAF_NODES,
        min_samples_leaf=HGB2_MIN_SAMPLE_LEAF,
        learning_rate=HGB2_LEARNING_RATE,
        l2_regularization=HGB2_L2_REGULARISATION, 
        max_iter=n, 
        early_stopping=False,
        random_state=42
    )
    model.fit(X_train, y_train)
    
    # Calcul des erreurs
    p_train = model.predict(X_train)
    p_test = model.predict(X_test)
    
    mse_tr = mean_squared_error(y_train, p_train)
    mse_te = mean_squared_error(y_test, p_test)
    
    # Stockage
    models_history[n] = model
    train_mse.append(mse_tr)
    test_mse.append(mse_te)
    iter_num.append(n)

# 3. Trouver l'itération optimale (le minimum du MSE de Test)
best_index = np.argmin(test_mse)
best_n = iter_num[best_index]
best_model = models_history[best_n]

print(f"Le modèle optimal est à n_iter = {best_n} avec un MSE de {test_mse[best_index]:.4f}")

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,5),constrained_layout=True)

ax.plot(iter_num,train_mse,color='b',label="train_mean_squared_error"  )
ax.plot(iter_num,test_mse ,color="r",label="test_mean_squared_error"  )
ax.set_xlabel("Iteration")
ax.set_ylabel("training error")
ax.set_title("training error")
ax.grid()
ax.legend()
ax.axvline(best_n,c="k")
plt.suptitle(suptitle2)
plt.show()


In [ ]:
# Calcul des predictions 
ypred_train_best = best_model.predict(X_train)
ypred_test_best = best_model.predict(X_test)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,6),constrained_layout=True)
ax.scatter(y_train,ypred_train_best,color="b",alpha=0.1)
ax.scatter(y_test,ypred_test_best,color="r",alpha=0.5)
ax.set_xlabel("$Y_{true}$")
ax.set_ylabel("$Y_{pred}$")
ax.plot([-20,20],[-20,20],lw=1,color="k")
ax.set_aspect("equal")
ax.grid()
ax.set_title(suptitle1)

In [ ]:
# Prédictions avec le modèle optimal
y_train_optimal = best_model.predict(X_train)
y_test_optimal = best_model.predict(X_test)

plt.figure(figsize=(12, 10))

# Plot pour le TEST
plt.scatter(y_test, y_test_optimal, alpha=0.5, color='coral', label='Test Data')
# Plot pour le TRAIN
plt.scatter(y_train, y_train_optimal, alpha=0.3, color='royalblue', label='Train Data')

# La ligne de perfection (Y = X)
limit = max(max(y_test), max(y_test_optimal))
plt.plot([0, limit], [0, limit], color='black', linestyle='--', label='Perfect Prediction')
plt.plot([-limit, 0], [-limit, 0], color='black', linestyle='-.')
plt.title(f"Y_pred vs Y_true (Optimal Iterations: {best_n})")
plt.xlabel("Valeurs Réelles (Y_true)")
plt.ylabel("Prédictions (Y_pred)")
plt.gca().set_aspect('equal', adjustable='box')
plt.legend()
plt.grid(True)
plt.suptitle(suptitle2)
plt.show()
